In [1]:
%%capture
import pandas as pd
import numpy as np
import re


# Thực hiện chung

In [2]:
# Đọc file Json, chuyển từ CSDL SQL trong MySQL sang File Json ổn định hơn và ít bị lỗi như chuyển sang File csv
df_sv = pd.read_json('raw_data/uit_sinhvien.json')
df_diem = pd.read_json('raw_data/uit_diem.json')
df_monhoc = pd.read_json('raw_data/uit_monhoc.json')
df_ctdaotao = pd.read_json('raw_data/uit_ctdaotao.json')


In [3]:
df_list1=df_monhoc[df_monhoc['trangthai'] ==1]
List1=df_list1['mamh'].to_list()

def Tao_listmontd(mondangxet,montuongduong):
    List2=montuongduong.split(',')
    if List2==['']:
        List2=[mondangxet]
        return List2
    List2.append(mondangxet)
    return List2

df_monhoc['List_montd']=df_monhoc.agg(lambda x: Tao_listmontd(x['mamh'],x['mh_td']),axis=1)

def Chon_monconmo(List2):
    #Cần lưu ý các môn như IS214,IS101
    #len(List2)==1 Trường hợp môn này ko có môn tương đương. Vì nếu ko có môn tương đương thì cũng có tên của chính nó trong List2.
    if len(List2)!=1:
        def fill_monconhoc(MoncanKT):
            return (MoncanKT in List1 )

        List3 = list(filter(lambda x: fill_monconhoc(x),List2))
        # Chọn thằng đứng đầu
        List3.sort()
        # Nếu List3 == [] chứng tỏ môn này có môn tương đương. Nhưng tất cả các môn tương đương đều bị đóng
        if List3 != []:
            return List3[0]
        # Chọn 1 thằng đại diện, ta đã biết được là các môn trong List2 này ko môn nào còn mở.
        List2.sort()
        return List2[0]
        
    return List2[0]

df_monhoc['Mondaidien']=df_monhoc.agg(lambda x: Chon_monconmo(x['List_montd']),axis=1)

Dict1={}
def Tao_Dict1(mamhcu,Mondaidien,Dict1):
    # mamhcu là khóa vì dùng nó để giải quyết thay thế cho các môn học đã cũ và khóa là duy nhất
    List_mamhcu=mamhcu.split(',')
    for i in List_mamhcu:
        Dict1[i]=Mondaidien
    return

df_Dict1=df_monhoc[df_monhoc['mamhcu'] !='']
df_Dict1.agg(lambda x: Tao_Dict1(x['mamhcu'],x['Mondaidien'],Dict1),axis=1)

def Doi_mamhcu(x):
    if x in Dict1.keys():
        x=Dict1[x]
    return x

df_monhoc['Mondaidien']=df_monhoc['Mondaidien'].apply(Doi_mamhcu)

df_monhoc.columns=['id', 'mamh', 'tenmh', 'tenta', 'loaimh', 'sotinchi', 'sotc_lt',
       'sotc_th', 'hedt_monhoc', 'khoaql', 'hinhthucth', 'mamhcu', 'mh_td',
       'mh_truoc', 'mh_tq', 'tldanhgia_qt', 'tldanhgia_th', 'tldanhgia_gk',
       'tldanhgia_ck', 'htthi_lt_gk', 'htthi_lt_gk_tg', 'htthi_lt_ck',
       'htthi_lt_ck_tg', 'htthi_th_ck', 'trangthai', 'ngonngu', 'ghichu',
       'dcid', 'List_montd', 'Mondaidien']

df_sv.columns=['id', 'masv', 'uid', 'hoten', 'ngaysinh', 'gioitinh', 'noisinh', 'cmnd',
       'cmnd_ngaycap', 'cmnd_noicap', 'lopsh', 'khoa', 'hedt_sv', 'khoahoc',
       'chuyennganh', 'chuyennganh2', 'nganh_tt', 'tinhtrang', 'email',
       'email_canhan', 'email_alias', 'dienthoai', 'dantoc', 'tongiao',
       'xuatthan', 'doituong', 'doituong_chitiet', 'khuvuc', 'vungmien',
       'chinhsach']

df_monhoc_selected=df_monhoc[['mamh','Mondaidien','sotc_lt','sotc_th','hinhthucth','khoaql','hedt_monhoc','ngonngu']]
df_monhoc_selected['hedt_monhoc']=df_monhoc_selected['hedt_monhoc'].str.upper()
df_monhoc_selected['hinhthucth']=df_monhoc_selected['hinhthucth'].map('hinhthucth{}'.format, na_action='ignore')

df_monhoc_selected = df_monhoc_selected.astype({'hinhthucth': str})

df_diem_selected=df_diem[['masv','mamh','diem','sotc','hocky','namhoc','trangthai']]
df_diem_selected['hocky']=df_diem_selected['hocky'].map('hk{}'.format, na_action='ignore')
df_diem_selected['trangthai']=df_diem_selected['trangthai'].map('trangthai{}'.format, na_action='ignore')

df_diem_selected = df_diem_selected.astype({"hocky": str, "trangthai": str})

df_diem_monhoc=pd.merge(df_diem_selected,df_monhoc_selected,on='mamh')
# Đã bỏ mamh
df_diem_monhoc=df_diem_monhoc[['masv','Mondaidien','diem','sotc','trangthai'
                               ,'sotc_lt','sotc_th','hinhthucth','ngonngu'
                               ,'khoaql','hedt_monhoc','hocky','namhoc']]

df_sv_selected=df_sv[['masv','gioitinh','noisinh','ngaysinh','cmnd_noicap','cmnd_ngaycap'
                      ,'lopsh','khoa','hedt_sv','khoahoc','chuyennganh','chuyennganh2','nganh_tt','dantoc','tongiao',
                     'xuatthan','doituong','doituong_chitiet','khuvuc']]

df_sv_selected['gioitinh']=df_sv_selected['gioitinh'].replace({1:'nam',0:'nu'})
df_sv_selected['dantoc']=df_sv_selected['dantoc'].map('dantoc{}'.format, na_action='ignore')
df_sv_selected['tongiao']=df_sv_selected['tongiao'].map('tongiao{}'.format, na_action='ignore')
df_sv_selected['xuatthan']=df_sv_selected['xuatthan'].map('xuatthan{}'.format, na_action='ignore')

df_sv_selected = df_sv_selected.astype({"gioitinh": str, "dantoc": str, "tongiao": str,"xuatthan": str})

df_tonghop3bang=pd.merge(df_sv_selected,df_diem_monhoc,on='masv')

list_cackhoa=df_tonghop3bang['khoahoc'].unique()
soluongcackhoa=df_tonghop3bang['khoahoc'].nunique()

print(list_cackhoa)
print(soluongcackhoa)

<ipython-input-3-a2be2feb4b42>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_monhoc_selected['hedt_monhoc']=df_monhoc_selected['hedt_monhoc'].str.upper()
<ipython-input-3-a2be2feb4b42>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_monhoc_selected['hinhthucth']=df_monhoc_selected['hinhthucth'].map('hinhthucth{}'.format, na_action='ignore')
<ipython-input-3-a2be2feb4b42>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

[ 9 10 11 12 13 14 15]
7


In [4]:
df_danhtinh=pd.DataFrame()
df_Final=pd.DataFrame()
soluong_0=pd.DataFrame()
soluong_1=pd.DataFrame()
soluong_2=pd.DataFrame()
khoa=[]
Cacnhan=[]
Thongke_Ketquadudoan=[]

In [5]:
def Xuly_dudoan(cackhoa_canchon): 
    global df_tonghop3bang
    global df_ctdaotao
    global df_danhtinh
    global df_Final
    global soluong_0
    global soluong_1
    global soluong_2
    global khoa
    global Cacnhan
    global Thongke_Ketquadudoan
    
    Nam1={}
    #Tạo quy đổi giữa khóa và năm học thứ nhất, năm học thứ 2.
    def Quydoi_khoa_nam1(khoa):
        khoa=int(khoa)
        return 2006+(khoa-1)

    for khoa in cackhoa_canchon:
        Nam1[khoa]=Quydoi_khoa_nam1(khoa)


    df_nam1_all=pd.DataFrame()

    for khoa in cackhoa_canchon:
        # Lấy thông tin 1 năm học đầu tiên ứng với sinh viên từng Khóa.
        df_nam1=df_tonghop3bang[(df_tonghop3bang.khoahoc==khoa)&
                                 (df_tonghop3bang.namhoc < Nam1[khoa]+1)
                               &(df_tonghop3bang.namhoc > Nam1[khoa]-1)]

        df_nam1_all=pd.concat([df_nam1_all,df_nam1],ignore_index=True)


    df_mienthi=pd.DataFrame()
    # Tách th miễn thi và hoãn thi => Để tính số tc cộng thêm thôi.
    df_mienthi=df_nam1_all[(df_nam1_all['trangthai']=='trangthai4')|(df_nam1_all['trangthai']=='trangthai5')]
    df_mienthi=df_mienthi.groupby(['masv','khoahoc','namhoc'], as_index=False).aggregate(sotc_thdb=pd.NamedAgg(column="sotc", aggfunc="sum"))
    df_muctieu1=pd.DataFrame()
    df_muctieu1=df_nam1_all[(df_nam1_all.trangthai!='trangthai0') & (df_nam1_all.trangthai!='trangthai4')
                             &(df_nam1_all.trangthai!='trangthai5')]


    df_muctieu1['Diem*Tc']= df_muctieu1.agg(lambda x: x['diem']*x['sotc'],axis=1)

    df_muctieu1=df_muctieu1.groupby(['masv','khoahoc','namhoc'], as_index=False).aggregate({'Diem*Tc':'sum', 'sotc': 'sum'})

    def Tinh_DTB(Diem_Tc,Tongsotc):
        if Tongsotc==0:
            return 0
        return (Diem_Tc/Tongsotc)
    # Để dưới dạng số để có thể xác định là có rớt hạng hay không.
    def Tinh_Loaihocluc(DTBnam):
        if DTBnam>=7.5:
            return 2
        elif DTBnam>=6.0:
            return 1
        else:
            return 0

    df_muctieu1['DTBnam']=df_muctieu1.agg(lambda x: Tinh_DTB(x['Diem*Tc'],x['sotc']),axis=1)
    df_muctieu1['Loaihoclucnam']=df_muctieu1.agg(lambda x: Tinh_Loaihocluc(x['DTBnam']),axis=1)
    df_dienthieu_diem=df_muctieu1[['masv','Diem*Tc']]

    df_muctieu1=df_muctieu1.drop(columns=['namhoc','Diem*Tc'])
    df_muctieu1.columns=['masv','khoahoc','Tong_sotc','DTBnam'
                                   ,'Loaihoclucnam']

    # Merge lại để cộng số tín chỉ hoãn thi, miễn thi vào tổng tín chỉ
    df_muctieu1=pd.merge(df_muctieu1,df_mienthi,on=['masv','khoahoc'],how='left')
    df_muctieu1=df_muctieu1.drop(columns='namhoc')

    df_muctieu1['Tong_sotc'] = np.where(df_muctieu1.sotc_thdb.isnull(),
                            df_muctieu1.Tong_sotc, 
                            df_muctieu1.Tong_sotc+df_muctieu1.sotc_thdb)

    # df_muctieu1 = df_muctieu1.astype({"Tong_sotc": int})

    df_muctieu1=df_muctieu1.drop(columns='sotc_thdb')

    # Học kỳ 3 gộp với HK2
    df_muctieu2=df_nam1_all.copy()
    df_muctieu2['hocky']=df_muctieu2['hocky'].replace({'hk3':'hk2'})
    # Lọc những môn hoãn thi, miễn thi ra bảng riêng. Theo từng học kỳ
    df_mienthi_hk=pd.DataFrame()

    df_mienthi_hk=df_muctieu2[(df_muctieu2['trangthai']=='trangthai4')|(df_muctieu2['trangthai']=='trangthai5')]

    df_mienthi_hk=df_mienthi_hk.groupby(['masv','hocky','khoahoc'], as_index=False).aggregate(
        sotc_thdb=pd.NamedAgg(column="sotc", aggfunc="sum"))

    df_muctieu2=df_muctieu2[(df_muctieu2.trangthai!='trangthai0')&(df_muctieu2.trangthai!='trangthai4')&
                            (df_muctieu2.trangthai!='trangthai5')]
    df_muctieu2['Diem*Tc']= df_muctieu2.agg(lambda x: x['diem']*x['sotc'],axis=1)
    df_muctieu2=df_muctieu2.groupby(['masv','hocky','khoahoc'], as_index=False).aggregate({'Diem*Tc':'sum', 'sotc': 'sum'})

    df_muctieu2['DTB_HK']=df_muctieu2.agg(lambda x: Tinh_DTB(x['Diem*Tc'],x['sotc']),axis=1)
    df_muctieu2['Loaihocluc_HK']=df_muctieu2.agg(lambda x: Tinh_Loaihocluc(x['DTB_HK']),axis=1)

    # Merge lại để cộng số tín chỉ hoãn thi, miễn thi vào tổng tín chỉ
    df_muctieu2=pd.merge(df_muctieu2,df_mienthi_hk,on=['masv','hocky','khoahoc'],how='left')




    df_muctieu2['sotc'] = np.where(df_muctieu2.sotc_thdb.isnull(),
                            df_muctieu2.sotc, 
                            df_muctieu2.sotc+df_muctieu2.sotc_thdb)


    df_muctieu2=df_muctieu2.drop(columns='sotc_thdb')


    df_muctieu3=df_muctieu2.copy()

    def Daodau_DTB_hk(x):
        if x['hocky']== 'hk1' and x['DTB_HK']>=0 :
            x['DTB_HK']= -x['DTB_HK']
        return x['DTB_HK']

    def Daodau_Loaihocluc_hk(x):
        if x['hocky']== 'hk1' and x['Loaihocluc_HK']>=0 :
            x['Loaihocluc_HK']= -x['Loaihocluc_HK']
        return x['Loaihocluc_HK']

    df_muctieu3['DTB_HK']=df_muctieu3.agg(lambda x: Daodau_DTB_hk(x),axis=1)

    df_muctieu3['Loaihocluc_HK']=df_muctieu3.agg(lambda x: Daodau_Loaihocluc_hk(x),axis=1)

    df_muctieu3=df_muctieu3.groupby(['masv','khoahoc'], as_index=False).aggregate(    
        Diff_DTB_HK=pd.NamedAgg(column="DTB_HK", aggfunc="sum"),
        Diff_Loaihocluc_HK=pd.NamedAgg(column="Loaihocluc_HK", aggfunc="sum"))

    df_muctieu2=df_muctieu2.groupby(['masv','khoahoc'], as_index=False).aggregate({'hocky':list,'sotc':list,'DTB_HK': list,
                                                                                  'Loaihocluc_HK':list})


    column_names_df_muctieu2 = ['masv', 'khoahoc','DTB_HK1','Sotc_HK1','Loaihocluc_HK1','DTB_HK2','Sotc_HK2','Loaihocluc_HK2']

    def Tao_df_muctieu2Final(x):
        # Gán dữ liệu cho List
        A_row=[x['masv'],x['khoahoc']]
        if len(x['hocky'])==2:
            for i in range(len(x['hocky'])):
                A_row.append(float(x['DTB_HK'][i]))
                A_row.append(int(x['sotc'][i]))
                A_row.append(int(x['Loaihocluc_HK'][i]))
        elif len(x['hocky'])==1:
            #Nếu có sửa đổi giá trị thể hiện của Học kỳ thì sửa tại đây thôi.
            if x['hocky'][0] =='hk1':
                A_row.append(float(x['DTB_HK'][0]))
                A_row.append(int(x['sotc'][0]))
                A_row.append(int(x['Loaihocluc_HK'][0]))
                A_row.extend([None,None,None])
            else:
                A_row.extend([None,None,None])
                A_row.append(float(x['DTB_HK'][0]))
                A_row.append(int(x['sotc'][0]))
                A_row.append(int(x['Loaihocluc_HK'][0]))
        else:
            A_row.extend([None,None,None,None,None,None])

        # Tạo ra Series
        output_series = pd.Series(A_row, index=column_names_df_muctieu2)
        return output_series

    df_muctieu2Final=df_muctieu2.apply(Tao_df_muctieu2Final,axis=1)
    df_muctieu2Final=pd.merge(df_muctieu2Final,df_muctieu3,on=['masv','khoahoc'])
    Somon_canlay=19
    df_nam1_sub1=df_nam1_all.groupby(['masv','khoahoc'], as_index=False).aggregate({'Mondaidien':list, 
                                                                                'diem':list,
                                                                                'sotc': list,
                                                                                'trangthai': list,
                                                                                'sotc_lt': list,
                                                                                'sotc_th': list,
                                                                                'hinhthucth': list,
                                                                                'ngonngu': list,
                                                                                'khoaql': list,
                                                                                'hedt_monhoc': list,
                                                                                'hocky': list,
                                                                               })

    # Tạo tên các cột
    column_names = ["masv", "khoahoc"]
    for i in range(Somon_canlay):
        mamh_='mamh_'+str(i+1)
        diemmon_='diemmon_'+str(i+1)
        sotcmon_='sotcmon_'+str(i+1)
        trangthaimon_='trangthaimon_'+str(i+1)
        sotc_ltmon_='sotc_ltmon_'+str(i+1)
        sotc_thmon_='sotc_thmon_'+str(i+1)
        hinhthucthmon_='hinhthucthmon_'+str(i+1)
        ngonngumon_='ngonngumon_'+str(i+1)
        khoaqlmon_='khoaqlmon_'+str(i+1)
        hedt_monhoc_='hedt_monhoc_'+str(i+1)
        hocky_mon_='hocky_mon_'+str(i+1)
        column_names.extend([mamh_,diemmon_,sotcmon_,trangthaimon_,
                            sotc_ltmon_,sotc_thmon_,hinhthucthmon_,ngonngumon_,
                            khoaqlmon_,hedt_monhoc_,hocky_mon_])
    def Tao_df_muctieu4(x):
        # Gán dữ liệu cho List
        A_row=[x['masv'],x['khoahoc']]

        for i in range(len(x['Mondaidien'])):
            A_row.append(x['Mondaidien'][i])
            A_row.append(x['diem'][i])
            A_row.append(x['sotc'][i])
            A_row.append(x['trangthai'][i])
            A_row.append(x['sotc_lt'][i])
            A_row.append(x['sotc_th'][i])
            A_row.append(x['hinhthucth'][i])
            A_row.append(x['ngonngu'][i])
            A_row.append(x['khoaql'][i])
            A_row.append(x['hedt_monhoc'][i])
            A_row.append(x['hocky'][i])

        #Những cột còn thiếu so với khuôn dữ liệu chuẩn đã tạo ra     
        So_conthieu=len(column_names)-len(A_row)
        for i in range(So_conthieu):
            A_row.append('')
        # Tạo ra Series
        output_series = pd.Series(A_row, index=column_names)
        return output_series


    df_muctieu4=df_nam1_sub1.apply(Tao_df_muctieu4,axis=1)
    df_Final=pd.merge(df_muctieu4,df_muctieu2Final,on=['masv','khoahoc'])
    df_Final=pd.merge(df_Final,df_muctieu1,on=['masv','khoahoc'])
    df_Final=pd.merge(df_sv_selected,df_Final,on=['masv','khoahoc'])


    df_Final=df_Final.drop(columns='doituong_chitiet')

    def Laynam(x):
        if x=='':
            return 0
        else:
            return int(x[0:4])
    df_Final['ngaysinh']=df_Final['ngaysinh'].map(lambda x: Laynam(x))
    df_Final['cmnd_ngaycap']=df_Final['cmnd_ngaycap'].map(lambda x: Laynam(x))

    def Tinhtuoi_cmnd(x):
        output=x['cmnd_ngaycap']-x['ngaysinh']
        if output<=0:
            return 0
        else:
            return output
    df_Final['cmnd_ngaycap']=df_Final.agg(lambda x: Tinhtuoi_cmnd(x),axis=1)

    def Tinhtuoi_vaoDH(x):
        output=Nam1[int(x['khoahoc'])]-x['ngaysinh'] 
        if output<=0:
            return 0
        else:
            return output
    df_Final['ngaysinh']=df_Final.agg(lambda x: Tinhtuoi_vaoDH(x),axis=1)

    df_Final=df_Final.rename(columns={"ngaysinh": "TuoivaoDH", "cmnd_ngaycap": "TuoicapCMND"})


    # Chuẩn hóa lớp sh
    def Chuanhoa_lopsh(x):
        output = ''.join([i for i in x if not i.isdigit()])
        return output
    df_Final['lopsh']=df_Final['lopsh'].map(lambda x: Chuanhoa_lopsh(x))

    # Làm sạch thông tin chung
    def no_accent_vietnamese(s):
        s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
        s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
        s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
        s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
        s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
        s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
        s = re.sub('[íìỉĩị]', 'i', s)
        s = re.sub('[ÍÌỈĨỊ]', 'I', s)
        s = re.sub('[úùủũụưứừửữự]', 'u', s)
        s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
        s = re.sub('[ýỳỷỹỵ]', 'y', s)
        s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
        s = re.sub('đ', 'd', s)
        s = re.sub('Đ', 'D', s)
        return s

    df_Final['noisinh']=df_Final['noisinh'].str.lower().str.normalize('NFKC')
    df_Final['cmnd_noicap']=df_Final['cmnd_noicap'].str.lower().str.normalize('NFKC')

    def Xuly_kytu(x):
        if x.startswith('tỉnh '):
            x = x[5:]
        if x.startswith('thành phố '):
            x = x[10:]
        if x.startswith('tp.'):
            x = x[3:]
        if x.endswith('city'):
            x = x[:-4]
        x=no_accent_vietnamese(x)

        if x.find('ho') != -1 and x.find('chi') != -1 and x.find('minh') != -1:
            x='hochiminh'
        if x.find('dac lac') != -1 or x.find('daklak')!= -1 :
            x='dak lak'
        if x.find('dac nong') != -1 or x.find('daknong')!= -1:
            x='dak nong'

        return x
    df_Final['noisinh'] = df_Final['noisinh'].map(Xuly_kytu)
    df_Final['cmnd_noicap'] = df_Final['cmnd_noicap'].map(Xuly_kytu)


    mode_noisinh=df_Final.noisinh.mode()[0]

    def Impute_noisinh(x,mode_noisinh):
        if x['noisinh']==''and x['cmnd_noicap']!='':
            x['noisinh']=x['cmnd_noicap']
            return x['noisinh']
        if x['noisinh']=='' and x['cmnd_noicap']=='':
            return mode_noisinh
        return x['noisinh']
    
    def Impute_cmnd_noicap(x):
        if x['noisinh']!=''and x['cmnd_noicap']=='':
            x['cmnd_noicap']=x['noisinh']
            return x['cmnd_noicap']

        return x['cmnd_noicap']


    df_Final['noisinh']=df_Final.apply(lambda x:Impute_noisinh(x,mode_noisinh),axis=1)
    df_Final['cmnd_noicap']=df_Final.apply(Impute_cmnd_noicap,axis=1)

    def Impute_thongtinchung_conlai(x):
        if x['gioitinh']=="":
            x['gioitinh']=df_Final.gioitinh.mode()[0]
        if x['TuoivaoDH']=="":
            x['TuoivaoDH']=df_Final.TuoivaoDH.mode()[0]
        if x['TuoicapCMND']=="":
            x['TuoicapCMND']=df_Final.TuoicapCMND.mode()[0]
        if x['TuoicapCMND']<14:
            x['TuoicapCMND']=14
        if x['nganh_tt']=="":
            x['nganh_tt']='khongco'
        if x['doituong']=="":
            x['doituong']='khongco'
        return x

    df_Final=df_Final.apply(Impute_thongtinchung_conlai,axis=1)

    df_ctdaotao_selected=df_ctdaotao[(df_ctdaotao.hocky==1) | (df_ctdaotao.hocky==2)
                             |(df_ctdaotao.hocky==3)]
    # nganh,hedt,khoahoc là những khóa sẽ kết với bảng dữ liệu chính
    df_ctdaotao_selected=df_ctdaotao_selected.groupby(['nganh','khoadt','hedt'], as_index=False).aggregate({'sotc':'sum'})
    df_ctdaotao_selected=df_ctdaotao_selected.rename(columns={"nganh": "chuyennganh", "khoadt": "khoahoc",
                                                             "hedt": "hedt_sv","sotc":"sotc_quydinh"})

    df_Final=pd.merge(df_Final,df_ctdaotao_selected,on=['chuyennganh','khoahoc','hedt_sv'],how='left')

    sotc_toithieu=28
    df_Final['sotc_quydinh'] = df_Final['sotc_quydinh'].where(df_Final['sotc_quydinh'].notnull(),sotc_toithieu)

    # Tích hợp bảng dữ liệu để có điểm mà tính trung bình theo luật mới điền vào cho các môn thiếu.
    df_dienthieu_diem
    df_Final=pd.merge(df_Final,df_dienthieu_diem,on='masv',how='left')

    Chon_gt=-1

    def dienthieu_monhoc(x,Chon_gt):
        for i in range(Somon_canlay):
            mamh_='mamh_'+str(i+1)
            diemmon_='diemmon_'+str(i+1)
            sotcmon_='sotcmon_'+str(i+1)
            trangthaimon_='trangthaimon_'+str(i+1)
            sotc_ltmon_='sotc_ltmon_'+str(i+1)
            sotc_thmon_='sotc_thmon_'+str(i+1)
            hinhthucthmon_='hinhthucthmon_'+str(i+1)
            ngonngumon_='ngonngumon_'+str(i+1)
            khoaqlmon_='khoaqlmon_'+str(i+1)
            hedt_monhoc_='hedt_monhoc_'+str(i+1)
            hocky_mon_='hocky_mon_'+str(i+1)

            if x[diemmon_]=="":
                if x['sotc_quydinh'] > x['Tong_sotc']:
                    x[diemmon_]=x['Diem*Tc']/x['sotc_quydinh']
                else:
                    x[diemmon_]=x['DTBnam']


            if x[sotcmon_]=="":
                x[sotcmon_]=Chon_gt
            if x[sotc_ltmon_]=="":
                x[sotc_ltmon_]=Chon_gt
            if x[sotc_thmon_]=="":
                x[sotc_thmon_]=Chon_gt

            if x[mamh_]=="":
                x[mamh_]='khongco'
            if x[trangthaimon_]=="":
                x[trangthaimon_]='khongco'
            if x[hinhthucthmon_]=="":
                x[hinhthucthmon_]='khongco'
            if x[ngonngumon_]=="":
                x[ngonngumon_]='khongco'
            if x[khoaqlmon_]=="":
                x[khoaqlmon_]='khongco'
            if x[hedt_monhoc_]=="":
                x[hedt_monhoc_]='khongco'
            if x[hocky_mon_]=="":
                x[hocky_mon_]='khongco'

        return x

    df_Final=df_Final.apply(lambda x: dienthieu_monhoc(x,Chon_gt),axis=1)
    
    
    
    df_danhtinh['masv']=df_Final.pop('masv')
    df_danhtinh['khoa']=df_Final['khoa']
    # Xóa các thuộc tính không cần thiết
    df_Final=df_Final.drop(columns='khoahoc')
    df_Final=df_Final.drop(columns='sotc_quydinh')
    df_Final=df_Final.drop(columns='Diem*Tc')

    # Do trong quá trình tính toán Pandas thì các giá trị ko có là np.nan chứ không phải chuỗi rỗng
    df_Final=df_Final.fillna(Chon_gt)

    chuoi=[]
    chuoi.extend(['gioitinh','noisinh','cmnd_noicap'
                  ,'lopsh','khoa','hedt_sv','chuyennganh','chuyennganh2','nganh_tt','dantoc','tongiao',
                 'xuatthan','doituong','khuvuc'])
    for i in range(Somon_canlay):
        mamh_='mamh_'+str(i+1)
        trangthaimon_='trangthaimon_'+str(i+1)
        hinhthucthmon_='hinhthucthmon_'+str(i+1)
        ngonngumon_='ngonngumon_'+str(i+1)
        khoaqlmon_='khoaqlmon_'+str(i+1)
        hedt_monhoc_='hedt_monhoc_'+str(i+1)
        hocky_mon_='hocky_mon_'+str(i+1)
        chuoi.extend([mamh_,trangthaimon_,
                            hinhthucthmon_,ngonngumon_,
                            khoaqlmon_,hedt_monhoc_,hocky_mon_])
    def bodau_chuoi(x):
        for i in chuoi:
            x[i]=no_accent_vietnamese(x[i])

        return x

    df_Final=df_Final.apply(bodau_chuoi,axis=1)

    for i in chuoi:
        df_Final[i]=df_Final[i].str.lower().str.normalize('NFKC')

    df_Final.columns= df_Final.columns.str.lower()    
    from sklearn.pipeline import Pipeline
    from sklearn.compose import ColumnTransformer
    from sklearn import preprocessing

    from sklearn.preprocessing import OneHotEncoder, StandardScaler

    from sklearn.ensemble import RandomForestClassifier

    from imblearn.combine import SMOTETomek
    from collections import Counter

    from joblib import load

    loaded_preprocessor = load("preprocessor_Final_huong13_Randomforest.dat")
    #out put là numpy array
    X_R=loaded_preprocessor.transform(df_Final)
    loaded_model = load("M_Final_huong13_Randomforest.dat")
    y_R_pred = loaded_model.predict(X_R)
    
    df_danhtinh['Ket qua du doan']=pd.DataFrame(y_R_pred)
    def Chuyen_doi_nhan(x):
        if x==2:
            return 'Xuatsac-Gioi-NuatrenKha, tu 7.5 tro len'
        elif x==1:
            return 'NuaduoiKha-Trungbinhkha, tu 6.0 den duoi 7.5'
        else:
            return 'DuoiTrungbinhkha, duoi 6.0'    
    df_danhtinh['Ket qua du doan']=df_danhtinh['Ket qua du doan'].apply(Chuyen_doi_nhan)
   

    Stats_Ketquadudoan=df_danhtinh['Ket qua du doan'].value_counts()
    Cacnhan=Stats_Ketquadudoan.index.to_list()
    Cacnhan[0]='Xuatsac-Gioi-NuatrenKha'
    Cacnhan[1]='NuaduoiKha-Trungbinhkha'
    Cacnhan[2]='DuoiTrungbinhkha'
    df_danhtinh['Soluong']=df_danhtinh.apply(lambda x:1,axis=1)
    khoa_Ketquadudoan=df_danhtinh.groupby(['khoa','Ket qua du doan'], as_index=False).aggregate(
        {'Soluong':'sum'})
    
    df_danhtinh=df_danhtinh.drop(columns='Soluong')
    
    soluong_0=khoa_Ketquadudoan[khoa_Ketquadudoan['Ket qua du doan']=='DuoiTrungbinhkha, duoi 6.0']
    soluong_0=soluong_0['Soluong'].to_list()

    soluong_1=khoa_Ketquadudoan[khoa_Ketquadudoan['Ket qua du doan']=='NuaduoiKha-Trungbinhkha, tu 6.0 den duoi 7.5']
    soluong_1=soluong_1['Soluong'].to_list()

    soluong_2=khoa_Ketquadudoan[khoa_Ketquadudoan['Ket qua du doan']=='Xuatsac-Gioi-NuatrenKha, tu 7.5 tro len']
    soluong_2=soluong_2['Soluong'].to_list()


    soluong_theokhoa=khoa_Ketquadudoan.groupby(['khoa'], as_index=False).aggregate(
        {'Soluong':'sum'})

    khoa=soluong_theokhoa['khoa'].to_list()
    Thongke_Ketquadudoan=Stats_Ketquadudoan.to_list()

In [6]:
from tkinter import *
from win32api import GetSystemMetrics
from tkinter.filedialog import Open
import tkinter.ttk as ttk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)

In [7]:
Width_screen=GetSystemMetrics(0)
Height_screen=GetSystemMetrics(1)
Width_app=(Width_screen*7)/8
Height_app=(Height_screen*6)/7
geometry = "%dx%d" % (Width_app,Height_app)
geometry

'1680x925'

In [8]:
Width_button=round(Width_app*0.015)
Height_button=round(Height_app*0.002)
borderwidth_frame=Height_button*0.5
Width_plot=Width_button*0.34
Height_plot=Height_button*4.5

In [9]:
# Width_button

In [10]:
# Height_button

In [11]:
# Width_plot

In [12]:
# Height_plot

In [13]:
class Example(Frame):
    def __init__(self, parent,list_cackhoa):
        Frame.__init__(self, parent, background='orange')
        self.parent = parent
        self.flag=0
        self.khoaduocchon=StringVar()
        self.frame6=Frame(self, bg='white',relief=RAISED, borderwidth=0)
        self.initUI(list_cackhoa)
        
    def initUI(self,list_cackhoa):
        self.parent.title("Khoanh vùng kết quả học tập của sinh viên năm hai dựa trên dữ liệu năm nhất")
        self.style = ttk.Style()
        self.style.theme_use("default")
        self.pack(fill=BOTH, expand=True)
        

        text_cackhoa=str()
        for i in list_cackhoa:
            text_cackhoa+=(str(i)+', ')

        text_cackhoa=text_cackhoa[:-2]

        
        frame = Frame(self,bg='white', relief=RAISED, borderwidth=0)
        frame.pack()
        label1= Label( frame, text='Nhập khóa cần khoanh vùng trong các khóa sau: '+text_cackhoa,bg='DarkOrange3',fg='white',
                      relief=RAISED,width = Width_button*3,height=round(Height_button*1.5))
        label1.config(font=("TkDefaultFont",round(Width_button*0.7)))
        label1.pack(side=TOP)
        
        frame0 = Frame(self,bg='white', relief=RAISED, borderwidth=0)
        frame0.pack(side=TOP)
        entry = Entry( frame0,textvariable = self.khoaduocchon,relief=RAISED,width = round(Width_button*0.17))
        
        entry.config(font=("TkDefaultFont",round(Width_button*0.7)))
        entry.pack()

        frame1 = Frame(self,bg='gray10', relief=RAISED, borderwidth=borderwidth_frame)
        frame1.pack(side=TOP)
        button_predict = Button(master = frame1, 
                            command = self.Khoanhvung_khoa,
                            height=Height_button,
                            width=Width_button,
                            text = "Xác nhận",
                            bg='gainsboro'
                       )
        button_predict.config(font=("TkDefaultFont",round(Width_button*0.5)))
        button_predict.pack()
        

        
        frame4=Frame(self, relief=RAISED, borderwidth=0)
        frame4.pack(side=TOP)
        
        label2 = Label( frame4, text='Biểu đồ kết quả khoanh vùng',bg='DarkOrange3',fg='white',
                      relief=RAISED,width = round(Width_button*3),height=round(Height_button*0.5))
        label2.config(font=("TkDefaultFont",round(Width_button*0.7)))
        label2.pack()
        
        frame5=Frame(self,bg='gray10', relief=RAISED, borderwidth=round(borderwidth_frame))
        frame5.pack(side=BOTTOM)
    
        button_printreport=Button(master = frame5, 
                                    command = self.In_ketqua,
                                    height=round(Height_button),
                                    width=round(Width_button),
                                    text = "In tập tin kết quả",
                                  bg='gainsboro')
        button_printreport.config(font=("TkDefaultFont",round(Width_button*0.5)))            
        button_printreport.pack()
        
        self.frame6.pack(fill=BOTH,expand=True)

    def plot(self):
        frame2 = Frame(self, relief=RAISED, borderwidth=round(borderwidth_frame*0.5))
        frame2.pack(side=LEFT,fill=None)
        
        frame3 = Frame(self, relief=RAISED, borderwidth=round(borderwidth_frame*0.5))
        frame3.pack(side=RIGHT,fill=None)
  
        x_plot1 = Cacnhan
        y_plot1 = Thongke_Ketquadudoan

        fig1 = Figure(figsize = (Width_plot, Height_plot))
        plot1 = fig1.add_subplot(111)

        # creating the bar plot
        plot1.bar(x_plot1, y_plot1, color ='maroon',
                width =Height_button*0.1)

        # plt.xlabel()
        plot1.set_ylabel("Số lượng sinh viên")
        plot1.set_title("Khoanh vùng kết quả học tập của sinh viên năm 2")
        
        # creating the Tkinter canvas
        # containing the Matplotlib figure
        canvas1 = FigureCanvasTkAgg(fig1,
                                   master = frame2)  
        canvas1.draw()

        # placing the canvas on the Tkinter window
        canvas1.get_tk_widget().pack()
        
        barWidth = Width_button*0.01
        fig2 = Figure(figsize = (Width_plot, Height_plot))
        plot2 = fig2.add_subplot(111)
        # set height of bar
        Xuatsac_Gioi_NuatrenKha = soluong_2
        NuaduoiKha_Trungbinhkha = soluong_1
        DuoiTrungbinhkha = soluong_0

        # Set position of bar on X axis
        br1 = np.arange(len(Xuatsac_Gioi_NuatrenKha))
        br2 = [x + barWidth for x in br1]
        br3 = [x + barWidth for x in br2]

        # Make the plot
        plot2.bar(br1, Xuatsac_Gioi_NuatrenKha, color ='r', width = barWidth,
                edgecolor ='grey', label ='Xuatsac-Gioi-NuatrenKha')
        plot2.bar(br2, NuaduoiKha_Trungbinhkha, color ='g', width = barWidth,
                edgecolor ='grey', label ='NuaduoiKha-Trungbinhkha')
        plot2.bar(br3, DuoiTrungbinhkha, color ='b', width = barWidth,
                edgecolor ='grey', label ='DuoiTrungbinhkha')

        # Adding Xticks
        plot2.set_xlabel('Các khoa', fontweight ='bold', fontsize = 9)
        plot2.set_ylabel('Số lượng sinh viên', fontweight ='bold', fontsize = 9)
        plot2.set_xticks([r + barWidth for r in range(len(Xuatsac_Gioi_NuatrenKha))])
        plot2.set_xticklabels(khoa)
        plot2.set_title("Khoanh vùng kết quả học tập của sinh viên năm 2")
        plot2.legend()

        # creating the Tkinter canvas
        # containing the Matplotlib figure
        canvas2 = FigureCanvasTkAgg(fig2,
                                   master = frame3)  
        canvas2.draw()

        # placing the canvas on the Tkinter window
        canvas2.get_tk_widget().pack()

    def Khoanhvung_khoa(self):
        khoa_cankhoanhvung=self.khoaduocchon.get()
#         self.khoaduocchon.set("")
        khoa_cankhoanhvung=[int(khoa_cankhoanhvung)]
        print(khoa_cankhoanhvung)
        Xuly_dudoan(khoa_cankhoanhvung)
        self.plot()
        self.flag=1
    def In_ketqua(self):
        if self.flag==1:
            df_danhtinh.to_csv('df_danhtinh.csv')
    
    
# the main Tkinter window
window = Tk()
  
# setting the title 
window.title('Plotting in Tkinter')

# dimensions of the main window
window.geometry(geometry)

app = Example(window,list_cackhoa)  
# run the gui
window.mainloop()
